In [31]:
import math
import sys
from typing import List
from functools import lru_cache, reduce, cmp_to_key
from itertools import starmap, accumulate, chain, islice, product, tee, pairwise
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
from importlib import reload
reload(arrays)
reload(trees)
import arrays as ar
import trees as tr
from trees import Node
print()

In [32]:
# Function to Build Tree
def buildTree(s):
    # Corner Case
    if (len(s) == 0 or s[0] == "N"):
        return None

    # Creating list of strings from input
    # string after splitting by space
    ip = list(map(str, s.split()))

    # Create the root of the tree
    root = Node(int(ip[0]))
    size = 0
    q = deque()

    # Push the root to the queue
    q.append(root)
    size = size + 1

    # Starting from the second element
    i = 1
    while (size > 0 and i < len(ip)):
        # Get and remove the front of the queue
        currNode = q[0]
        q.popleft()
        size = size - 1

        # Get the current node's value from the string
        currVal = ip[i]

        # If the left child is not null
        if (currVal != "N"):
            # Create the left child for the current node
            currNode.left = Node(int(currVal))

            # Push it to the queue
            q.append(currNode.left)
            size = size + 1
        # For the right child
        i = i + 1
        if (i >= len(ip)):
            break
        currVal = ip[i]

        # If the right child is not null
        if (currVal != "N"):
            # Create the right child for the current node
            currNode.right = Node(int(currVal))

            # Push it to the queue
            q.append(currNode.right)
            size = size + 1
        i = i + 1
    return root


In [40]:
def nodes_at_distance(root: Node, target: int, k: int) -> List[int]:
    "Return nodes from tree `root` at distance `k` from `target` node."
    result = []
    def descend(r: Node, kn: int):
        if not r or kn < 0: return
        #print(f'd {r.data} @ {kn}')
        if kn == 0:
            result.append(r.data)
            return
        descend(r.left, kn -1)
        descend(r.right, kn -1)

    def search(r: Node):
        if not r: return None
        #print(f'd {r.data}')
        if r.data == target:
            descend(r, k)
            return 1
        d = search(r.left)
        if d:
            descend(r.right, k-d-1)
        else:
            d = search(r.right)
            if d:
                descend(r.left, k-d-1)
        if k == d:
            result.append(r.data)
            return None
        return d + 1 if d else None
    search(root)
    return sorted(result)


In [41]:
tree = buildTree('12 9 7 13 2 3 4 6 N N N 5 11 N N N N N N N N')
nodes_at_distance(tree, 4, 1)

d 12
d 9
d 13
d 6
d 2
d 7
d 3
d 5
d 11
d 4
d 4 @ 1


[7]

In [20]:
tree